# Mirror for extinction Cerenkov detector

This will calculate the geometry for the Cerenkov mirror and export it as an STL file, which can be imported into Geant4 using CADMesh.

Although it's a little non-standard, this file will be kept and run in the repository area and only the STL file will be copied to the build area.  This is to prevent possible confusion caused by multiple copies.


In [8]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # needed for 3D
import trimesh
import stl

# set up some parameters
focus_filename = 'focus.txt' # text file containing the focus position
stl_filename = 'Mirror.stl'  # File that STL will be written to
n_radiator = 1.33  # refractive index of radiator (water)
l_radiator = 60.   # length of radiator [mm]
z_mirror = 500.    # z position of top of radiator [mm]
mirror_height = 50. # Mirror full height [mm]
mirror_thickness = 5 # Mirror thickness [mm]
n_segments = 200   # Number of segments in half the mirror
K = 8.             # Kinetic energy of beam [GeV]
m = .938           # mass of proton [GeV/c^2]

# do some calculations
beta = np.sqrt((K+m)**2-m**2)/(K+m)
print(f"beta = {beta:.4f}")
ang = np.acos(1/n_radiator/beta)
print(f"Cerenkov angle = {ang:.4f} radians ({ang*180/np.pi:.4f} degrees)")

# Calculate the maximum rho
delta_z=z_mirror-l_radiator/2.
rho = delta_z*np.tan(ang)
# calcualate the maxium theta.  This won't be quite right. 
# We'll trim it at the end to keep section from overlapping
theta_max = np.asin(np.tan(ang)/np.sqrt(2.))
print(f"rho = {rho:.4f} mm, theta_max = {theta_max*180/np.pi:.4f} degrees")

# Generate the angles for half the mirror and put them into a panda table
theta = np.linspace(0.,theta_max,n_segments)
points = pd.DataFrame({'theta':theta})

# Now calculate the center points of each segment
points['z'] = l_radiator/2. + delta_z*np.cos(points['theta'])
points['x'] = delta_z*np.sin(points['theta'])
points['y'] = np.sqrt(rho**2-points['x']**2)

# put the detector between the lowest and highest point
y_detector = (points['y'].max()+points['y'].min())/2.
# Store the focus location so it can be read out by the C++ program
x_focus = 0.
y_focus = y_detector
z_focus = l_radiator/2.
print(f"Focus location: x={x_focus:.2f} mm, y={y_focus:.2f} mm, z={z_focus:.2f} mm.")
print(f"Writing focus location to file {focus_filename}...")
with open(focus_filename,"w") as f:
    f.write(f"{x_focus} {y_focus} {z_focus}")


# Angle in the Z Y plane from the source to the mirror point
points['ang_source'] = np.atan(points['y']/(points['z']-l_radiator/2.))
# angle in the Z Y plane from the mirror point to the detector
points['ang_detector'] = np.atan((y_detector-points['y'])/(points['z']-l_radiator/2.))
# mirror tilt in the Z Y plane (positive is forward)
points['alpha'] = (points['ang_detector']-points['ang_source'])/2.
# Now calculate the top and bottom points
dy = mirror_height/2.
points['y_top'] = points['y']+dy
points['y_bottom'] = points['y']-dy
points['z_top'] = points['z'] + dy*np.tan(points['alpha'])
points['z_bottom'] = points['z'] - dy*np.tan(points['alpha'])
# Eliminate potential overlaps
points['phi_bottom']=np.atan(points['x']/points['y_bottom'])
# Cut off points that could potentially overlap
points = points[points['phi_bottom']<.95*np.pi/4.]

# Mirror the points
points_copy = points[points['theta']!=0.].copy()
points_copy['x'] = -points_copy['x']
points_copy['theta'] = -points_copy['theta']
# Append them and resort it
points = pd.concat([points_copy, points],ignore_index=True)
points = points.sort_values(by='theta')

# Now collect the top and the bottom points
# top points
tf = pd.DataFrame()
tb = pd.DataFrame()
tf['x'] = points['x']
tf['y'] = points['y_top']
tf['z'] = points['z_top']
tb = tf.copy()
tb['z'] += mirror_thickness
tf = tf[['x', 'y', 'z']].to_numpy()
tb = tb[['x', 'y', 'z']].to_numpy()
# bottom points
bf = pd.DataFrame()
bf['x'] = points['x']
bf['y'] = points['y_bottom']
bf['z'] = points['z_bottom']
bb = bf.copy()
bb['z'] += mirror_thickness
bf = bf[['x', 'y', 'z']].to_numpy()
bb = bb[['x', 'y', 'z']].to_numpy()
# Now interleave all four
#  -bottom front
#  -top front
#  -bottom back
#  -top back
vertices = np.empty((len(tf)*4,3),dtype=tf.dtype)
vertices[0::4] = bf
vertices[1::4] = tf
vertices[2::4] = bb
vertices[3::4] = tb
# Now create the faces.
#  There will be 8 faces for every step, plus 2 at the left end and 2 at the right
faces = np.empty(((len(tf)-1)*8+4,3),dtype=int)

# Now we'll go through and build the faces
ioffs = 0   # pointer to current set of vertices
joffs = 0   # pointer to current set of faces
for i in range(len(tf)-1):
    # There's probably a cuter way to do this, but I want one that works
    ibf = ioffs+0   # bottom front
    itf = ioffs+1   # top front
    ibb = ioffs+2   # bottom back
    itb = ioffs+3   # top back
    inbf = ioffs+4  # next bottom fronf
    intf = ioffs+5  # next top front
    inbb = ioffs+6  # next bottom back
    intb = ioffs+7  # next top back
    # build the two front faces
    faces[joffs] = [ibf,inbf,intf]  # bottom front, next bottom front, next top front
    faces[joffs+1] = [ibf,intf,itf] # botton front, next top front, top front
    # build the back two fraces
    faces[joffs+2] = [inbb,ibb,intb] # next bottom back, bottom back, next top back
    faces[joffs+3] = [ibb,itb,intb]  # bottom back, top back, next top back
    # build the top faces
    faces[joffs+4] = [itf,intf,intb] # top front, next top front, next top back
    faces[joffs+5] = [itf,intb,itb]  # top front, next top back, top back
    # build the bottom faces
    faces[joffs+6] = [inbf,ibf,inbb] # next bottom front, bottom front, next bottom back
    faces[joffs+7] = [ibf,ibb,inbb] # bottom front, bottom back, next bottom back
    ioffs += 4
    joffs += 8
# now build the left side
ibf = 0
itf = 1
ibb = 2
itb = 3
faces[joffs] = [ibb,ibf,itf]   # bottom back, bottom front, top front
faces[joffs+1] = [itf,itb,ibb] # top front, top back, bottom back
# build right side
ibf = -4
itf = -3
ibb = -2
itb = -1
faces[joffs+2] = [ibf,ibb,itb] # bottom front, bottom back, top back
faces[joffs+3] = [itb,itf,ibf] # top back, top front, bottom front

# Now make  a tesselated surface
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# Make sure it's watertight
if mesh.is_watertight:
    print("The mesh is watertight.")
else:
    print("The mesh is NOT watertight.")

# for some reason this is necessary, even though I was careful to get the winding right.
# Otherwise it won't reflect!
mesh.fix_normals()

# Have to write it as an ascii file or CADMesh can't read it.
print(f"Writing solid to {stl_filename}...")

with open(stl_filename, 'w') as f:
    f.write(trimesh.exchange.stl.export_stl_ascii(mesh))

beta = 0.9945
Cerenkov angle = 0.7135 radians (40.8824 degrees)
rho = 406.8740 mm, theta_max = 37.7440 degrees
Focus location: x=0.00 mm, y=347.29 mm, z=30.00 mm.
Writing focus location to file focus.txt...
The mesh is watertight.
Writing solid to Mirror.stl...
